In [252]:
import pandas as pd
import cv2
import os

from PIL import Image
import pytesseract
import cv2
import numpy as np

In [210]:

r_data = pd.read_csv('/home/scar3crow/Downloads/invoice_data_1/via_region_data.csv')
num_obj = r_data['region_count'][0] # number of objects in each photo
r_data.drop(r_data.columns[[1, 2, 3, 4]], axis=1, inplace=True) # reduce unnecessary columns
r_data.sort_values(by=['#filename'], ascending=True) # Sorting based on image-id
num_images = r_data["#filename"].nunique() # Find out number of unique images

print('Number of classes = ', num_obj)
print('Number of unique images = ', num_images)
r_data.head()

Number of classes =  7
Number of unique images =  30


,#filename,region_shape_attributes,region_attributes
0,30.jpg,"{""name"":""rect"",""x"":91,""y"":125,""width"":394,""hei...","{""text"":""vendor""}"
1,30.jpg,"{""name"":""rect"",""x"":79,""y"":175,""width"":426,""hei...","{""text"":""buyer""}"
2,30.jpg,"{""name"":""rect"",""x"":38,""y"":83,""width"":136,""heig...","{""text"":""invoice""}"
3,30.jpg,"{""name"":""rect"",""x"":412,""y"":88,""width"":76,""heig...","{""text"":""inv_date""}"
4,30.jpg,"{""name"":""rect"",""x"":94,""y"":253,""width"":74,""heig...","{""text"":""po""}"


In [213]:
type(r_data['#filename'][0])

str

In [214]:
# Making a dataframe for Image_id, x, y, width, height and class

x = []
y = []
width = []
height = []
obj_class = []


for i in range(len(r_data)):
    
    r_size = r_data.values[i, 1][1:(len(r_data.values[i, 1])-1)]
    r_size_par = r_size.split(",")
    
    x.append(int("".join(filter(str.isdigit, r_size_par[1]))))
    y.append(int("".join(filter(str.isdigit, r_size_par[2]))))
    width.append(int("".join(filter(str.isdigit, r_size_par[3]))))
    height.append(int("".join(filter(str.isdigit, r_size_par[4]))))
    
    r_attribs = r_data.values[i, 2][1:(len(r_data.values[i, 2])-1)]
    r_attribs_par = r_attribs.split(':')[1]
    obj_class.append(r_attribs_par[1:(len(r_attribs_par)-1)])
    
x_values = pd.Series(x)
y_values = pd.Series(y)
width_values = pd.Series(width)
height_values = pd.Series(height)
class_values = pd.Series(obj_class)

r_data.insert(loc=1, column='x', value=x_values)
r_data.insert(loc=2, column='y', value=y_values)
r_data.insert(loc=3, column='width', value=width_values)
r_data.insert(loc=4, column='height', value=height_values)
r_data.insert(loc=5, column='obj_class', value=class_values)

r_data.drop(r_data.columns[[6, 7]], axis=1, inplace=True) # reduce unnecessary columns

r_data.rename({'#filename': 'img_id'}, axis=1, inplace=True) # changing column name

r_data.head()

,img_id,x,y,width,height,obj_class
0,30.jpg,91,125,394,45,vendor
1,30.jpg,79,175,426,81,buyer
2,30.jpg,38,83,136,26,invoice
3,30.jpg,412,88,76,23,inv_date
4,30.jpg,94,253,74,19,po


In [207]:
# Check whether all images have got same number of classes

r_data.groupby('img_id').img_id.count()

img_id
1.jpg     7
10.jpg    7
11.jpg    7
12.jpg    7
13.jpg    7
14.jpg    7
15.jpg    7
16.jpg    7
17.jpg    7
18.jpg    7
19.jpg    7
2.jpg     7
20.jpg    7
21.jpg    7
22.jpg    7
23.jpg    7
24.jpg    7
25.jpg    7
26.jpg    7
27.jpg    7
28.jpg    7
29.jpg    7
3.jpg     7
30.jpg    7
4.jpg     7
5.jpg     7
6.jpg     7
7.jpg     7
8.jpg     7
9.jpg     7
Name: img_id, dtype: int64

In [233]:
# Making a list of image path

inv_directory = '/home/scar3crow/Downloads/INVOICES'  ## 'invoices' is a zip file of jpg images in ...../Downloads 
                                                        
inv_image = ['/home/scar3crow/Downloads/INVOICES/{}'.format(i) for i in os.listdir(inv_directory)] # making the list
inv_image.sort() # Sorting the list

len(inv_image)
inv_image[50]

'/home/scar3crow/Downloads/INVOICES/53.jpg'

In [251]:
## Creating train images by cropping original images as per x, y, width, height of bbox created in VGG I.A. tool

my_class_image = [] # List of class images with proper id
kount = 0

for i in range(len(inv_image)):
    
    id = inv_image[i].split('/')[-1] # finding the image_id
    indices = r_data.index[r_data['img_id'] == id].tolist() # making the list of indices of r_data where same image_id exists
    
    if len(indices) != 0:
        
        image = cv2.imread(inv_image[i]) ## Loading image
    
        kount = kount + 1
    
        for j in indices:
        
            image_crop = image[r_data['y'][j]:r_data['y'][j]+r_data['height'][j], r_data['x'][j]:r_data['x'][j]+r_data['width'][j]]
        
            class_sl_id = '/home/scar3crow/Downloads/train_invoice/' + id.split('.')[0] + '-' + str(r_data['obj_class'][j]) + str(kount) + '.jpg'
        
            my_class_image.append(class_sl_id)
            
            cv2.imwrite(class_sl_id, image_crop)

## To make all images of same size. Object boundary box selection to be done after this.

## i.e. after all images are of same size use VGG Image Annotation tool

In [161]:
# Check sizes of exiting images :

rows = []
columns = []
image_sl = []
df = pd.DataFrame()

for i in range(len(inv_image)):
    image = cv2.imread(inv_image[i]) ## Loading image
    height, width, _ = image.shape
    rows.append(height)
    columns.append(width)
    image_sl.append(inv_image[i])
    
row_values = pd.Series(rows)
col_values = pd.Series(columns)
image_num = pd.Series(image_sl)


df.insert(loc=0, column='image_serial', value=image_num)
df.insert(loc=1, column='rows', value=row_values)
df.insert(loc=2, column='columns', value=col_values)

df.head(10)


,image_serial,rows,columns
0,/home/scar3crow/Downloads/INVOICES/1.jpg,3504,2528
1,/home/scar3crow/Downloads/INVOICES/10.jpg,3504,2528
2,/home/scar3crow/Downloads/INVOICES/11.jpg,3504,2528
3,/home/scar3crow/Downloads/INVOICES/12.jpg,3504,2528
4,/home/scar3crow/Downloads/INVOICES/13.jpg,3504,2528
5,/home/scar3crow/Downloads/INVOICES/14.jpg,2736,1968
6,/home/scar3crow/Downloads/INVOICES/15.jpg,3504,2528
7,/home/scar3crow/Downloads/INVOICES/16.jpg,3504,2528
8,/home/scar3crow/Downloads/INVOICES/17.jpg,3504,2528
9,/home/scar3crow/Downloads/INVOICES/18.jpg,3504,2528


In [162]:
df[11:20]

,image_serial,rows,columns
11,/home/scar3crow/Downloads/INVOICES/2.jpg,3504,2528
12,/home/scar3crow/Downloads/INVOICES/20.jpg,3504,2528
13,/home/scar3crow/Downloads/INVOICES/21.jpg,3504,2528
14,/home/scar3crow/Downloads/INVOICES/22.jpg,3504,2528
15,/home/scar3crow/Downloads/INVOICES/23.jpg,3504,2528
16,/home/scar3crow/Downloads/INVOICES/24.jpg,3504,2528
17,/home/scar3crow/Downloads/INVOICES/25.jpg,768,557
18,/home/scar3crow/Downloads/INVOICES/26.jpg,768,557
19,/home/scar3crow/Downloads/INVOICES/27.jpg,768,557


In [163]:
df[21:30]

,image_serial,rows,columns
21,/home/scar3crow/Downloads/INVOICES/29.jpg,768,557
22,/home/scar3crow/Downloads/INVOICES/3.jpg,3504,2528
23,/home/scar3crow/Downloads/INVOICES/30.jpg,768,557
24,/home/scar3crow/Downloads/INVOICES/31.jpg,768,557
25,/home/scar3crow/Downloads/INVOICES/32.jpg,768,557
26,/home/scar3crow/Downloads/INVOICES/33.jpg,768,557
27,/home/scar3crow/Downloads/INVOICES/34.jpg,768,557
28,/home/scar3crow/Downloads/INVOICES/35.jpg,768,557
29,/home/scar3crow/Downloads/INVOICES/36.jpg,768,557


In [164]:
df[31:40]

,image_serial,rows,columns
31,/home/scar3crow/Downloads/INVOICES/37-B.jpg,768,557
32,/home/scar3crow/Downloads/INVOICES/37-C.jpg,768,557
33,/home/scar3crow/Downloads/INVOICES/38.jpg,768,554
34,/home/scar3crow/Downloads/INVOICES/39.jpg,768,554
35,/home/scar3crow/Downloads/INVOICES/4.jpg,3504,2528
36,/home/scar3crow/Downloads/INVOICES/40.jpg,768,554
37,/home/scar3crow/Downloads/INVOICES/41.jpg,768,554
38,/home/scar3crow/Downloads/INVOICES/42.jpg,768,554
39,/home/scar3crow/Downloads/INVOICES/43.jpg,768,554


In [165]:
df[41:50]

,image_serial,rows,columns
41,/home/scar3crow/Downloads/INVOICES/45.jpg,768,554
42,/home/scar3crow/Downloads/INVOICES/46.jpg,768,554
43,/home/scar3crow/Downloads/INVOICES/47.jpg,768,554
44,/home/scar3crow/Downloads/INVOICES/48.jpg,768,554
45,/home/scar3crow/Downloads/INVOICES/49.jpg,768,554
46,/home/scar3crow/Downloads/INVOICES/5.jpg,3504,2528
47,/home/scar3crow/Downloads/INVOICES/50.jpg,768,554
48,/home/scar3crow/Downloads/INVOICES/51.jpg,768,554
49,/home/scar3crow/Downloads/INVOICES/52.jpg,768,554


In [170]:
df[51:]

,image_serial,rows,columns
51,/home/scar3crow/Downloads/INVOICES/54.jpg,768,554
52,/home/scar3crow/Downloads/INVOICES/55.jpg,768,554
53,/home/scar3crow/Downloads/INVOICES/56.jpg,768,554
54,/home/scar3crow/Downloads/INVOICES/57.jpg,768,554
55,/home/scar3crow/Downloads/INVOICES/58.jpg,768,554
56,/home/scar3crow/Downloads/INVOICES/59.jpg,768,554
57,/home/scar3crow/Downloads/INVOICES/6.jpg,3504,2528
58,/home/scar3crow/Downloads/INVOICES/60.jpg,768,556
59,/home/scar3crow/Downloads/INVOICES/61.jpg,768,554
60,/home/scar3crow/Downloads/INVOICES/62.jpg,768,554


In [180]:
# Finding minimimum sizes :

i = df[['rows']].idxmin()
j = df[['columns']].idxmin()

print(i)
print(j)

print(df.loc[i])
print(df.loc[j])

rows    17
dtype: int64
columns    33
dtype: int64
                                 image_serial  rows  columns
17  /home/scar3crow/Downloads/INVOICES/25.jpg   768      557
                                 image_serial  rows  columns
33  /home/scar3crow/Downloads/INVOICES/38.jpg   768      554


In [181]:
# Resize to 768 X 554

for i in range(len(df)):
    
    original_image = cv2.imread(df['image_serial'][i])
    resized_image = cv2.resize(original_image, (554, 768)) ## Note width(column), height(row) format in resize
    cv2.imwrite(df['image_serial'][i], resized_image) ## writting or over-writting on the old image
    

In [185]:
# Checking whether all images are now of same size

rows = []
columns = []
image_sl = []
df = pd.DataFrame()

for i in range(len(inv_image)):
    image = cv2.imread(inv_image[i]) ## Loading image
    height, width, _ = image.shape
    rows.append(height)
    columns.append(width)
    image_sl.append(inv_image[i])
    
row_values = pd.Series(rows)
col_values = pd.Series(columns)
image_num = pd.Series(image_sl)


df.insert(loc=0, column='image_serial', value=image_num)
df.insert(loc=1, column='rows', value=row_values)
df.insert(loc=2, column='columns', value=col_values)

df.head(10)

# Now, let's go for boundary box selection with VGG Image Annotation Tool

,image_serial,rows,columns
0,/home/scar3crow/Downloads/INVOICES/1.jpg,768,554
1,/home/scar3crow/Downloads/INVOICES/10.jpg,768,554
2,/home/scar3crow/Downloads/INVOICES/11.jpg,768,554
3,/home/scar3crow/Downloads/INVOICES/12.jpg,768,554
4,/home/scar3crow/Downloads/INVOICES/13.jpg,768,554
5,/home/scar3crow/Downloads/INVOICES/14.jpg,768,554
6,/home/scar3crow/Downloads/INVOICES/15.jpg,768,554
7,/home/scar3crow/Downloads/INVOICES/16.jpg,768,554
8,/home/scar3crow/Downloads/INVOICES/17.jpg,768,554
9,/home/scar3crow/Downloads/INVOICES/18.jpg,768,554


In [183]:
df[11:20]

,image_serial,rows,columns
11,/home/scar3crow/Downloads/INVOICES/2.jpg,768,554
12,/home/scar3crow/Downloads/INVOICES/20.jpg,768,554
13,/home/scar3crow/Downloads/INVOICES/21.jpg,768,554
14,/home/scar3crow/Downloads/INVOICES/22.jpg,768,554
15,/home/scar3crow/Downloads/INVOICES/23.jpg,768,554
16,/home/scar3crow/Downloads/INVOICES/24.jpg,768,554
17,/home/scar3crow/Downloads/INVOICES/25.jpg,768,554
18,/home/scar3crow/Downloads/INVOICES/26.jpg,768,554
19,/home/scar3crow/Downloads/INVOICES/27.jpg,768,554


In [184]:
df.tail()

,image_serial,rows,columns
61,/home/scar3crow/Downloads/INVOICES/63.jpg,768,554
62,/home/scar3crow/Downloads/INVOICES/64.jpg,768,554
63,/home/scar3crow/Downloads/INVOICES/7.jpg,768,554
64,/home/scar3crow/Downloads/INVOICES/8.jpg,768,554
65,/home/scar3crow/Downloads/INVOICES/9.jpg,768,554


In [255]:
anchors = '1.08, 1.19, 3.42,4.41, 6.63,11.38, 9.42,5.11, 16.62,10.52'
anchors_1 = [float(anchors.strip()) for anchors in anchors.split(',')]

In [257]:
print(type(anchors))
anchors

<class 'str'>


'1.08, 1.19, 3.42,4.41, 6.63,11.38, 9.42,5.11, 16.62,10.52'

In [258]:
print(type(anchors_1))
anchors_1

<class 'list'>


[1.08, 1.19, 3.42, 4.41, 6.63, 11.38, 9.42, 5.11, 16.62, 10.52]